In [2]:
import pandas as pd

# Load the diagnoses_icd.csv file
df = pd.read_csv("~/Desktop/diagnoses_icd.csv")

# Ensure the icd_code column is treated as a string
df["icd_code"] = df["icd_code"].astype(str)

# Define ICD-10 codes of interest using regular expressions
major_depression_codes = ["F32", "F33"]
bipolar_disorder_codes = ["F25.0", "F30", "F31"]
persistent_mood_disorder_codes = ["F34"]
eating_disorder_codes = ["F50"]
schizophrenia_codes = ["F20"]


In [3]:
# Function to check if an ICD code matches the category
def match_icd(code, categories):
    return any(code.startswith(c) for c in categories)

# Filter the data based on ICD-10 codes
df_filtered = df[df["icd_code"].apply(lambda x: 
    match_icd(x, major_depression_codes) or
    match_icd(x, bipolar_disorder_codes) or
    match_icd(x, persistent_mood_disorder_codes) or
    match_icd(x, eating_disorder_codes) or
    match_icd(x, schizophrenia_codes))
]

# Count the frequency of each diagnosis
diagnosis_counts = df_filtered["icd_code"].value_counts()

# Display results
print(diagnosis_counts)

F329     41876
F319      6870
F32A      4367
F209      2217
F339      1118
         ...  
F3175        1
F3171        1
F3176        1
F348         1
F3011        1
Name: icd_code, Length: 68, dtype: int64


In [4]:
# Ensure the icd_code column is treated as a string
df["icd_code"] = df["icd_code"].astype(str)

# Define ICD-10 codes of interest using their prefixes
icd_categories = {
    "Major Depression": ["F32", "F33"],
    "Bipolar Disorder": ["F25", "F30", "F31"],
    "Persistent Mood Disorder": ["F34"],
    "Eating Disorder": ["F50"],
    "Schizophrenia": ["F20"]
}

# Function to classify ICD codes into categories
def classify_icd(code):
    for category, codes in icd_categories.items():
        if any(code.startswith(c) for c in codes):
            return category
    return None  # Ignore other diagnoses

# Apply classification to filter relevant diagnoses
df["diagnosis_category"] = df["icd_code"].apply(classify_icd)
df_filtered = df.dropna(subset=["diagnosis_category"])  # Keep only relevant diagnoses

# Group by diagnosis category and count occurrences
diagnosis_counts = df_filtered.groupby("diagnosis_category")["icd_code"].count()

# Display results
print(diagnosis_counts)


diagnosis_category
Bipolar Disorder            11101
Eating Disorder              1157
Major Depression            49471
Persistent Mood Disorder      209
Schizophrenia                2984
Name: icd_code, dtype: int64


In [5]:
# Filter for primary diagnoses (seq_num = 1)
df_primary = df[df["seq_num"] == 1]
df_primary["icd_code"] = df_primary["icd_code"].astype(str)

# Apply classification to filter relevant diagnoses
df_primary["diagnosis_category"] = df_primary["icd_code"].apply(classify_icd)
df_primary_filtered = df_primary.dropna(subset=["diagnosis_category"])  # Keep only relevant diagnoses

# Group by diagnosis category and count occurrences
primary_diagnosis_counts = df_primary_filtered.groupby("diagnosis_category")["icd_code"].count()

# Display results
print(primary_diagnosis_counts)


/var/folders/pc/mc5drrzd1875crsj_cnkc0b80000gn/T/ipykernel_58601/2299711856.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_primary["icd_code"] = df_primary["icd_code"].astype(str)


diagnosis_category
Bipolar Disorder            2572
Eating Disorder              213
Major Depression            5455
Persistent Mood Disorder      16
Schizophrenia                855
Name: icd_code, dtype: int64


/var/folders/pc/mc5drrzd1875crsj_cnkc0b80000gn/T/ipykernel_58601/2299711856.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_primary["diagnosis_category"] = df_primary["icd_code"].apply(classify_icd)


In [6]:
df_primary_filtered.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,diagnosis_category
1697,10002315,22798410,1,F329,10,Major Depression
5307,10006630,27341635,1,F329,10,Major Depression
5549,10007218,27767127,1,F329,10,Major Depression
5551,10007218,28348073,1,F329,10,Major Depression
5558,10007266,27967951,1,F332,10,Major Depression


In [7]:
import pandas as pd

# Load the patients.csv file to get anchor year group
patients_df = pd.read_csv("~/Desktop/patients.csv")

# Merge with primary diagnosis data
df_primary_filtered = df_primary_filtered.merge(patients_df, on="subject_id", how="left")

# Group by anchor_year_group and diagnosis_category, then count occurrences
anchor_year_summary = df_primary_filtered.groupby(["anchor_year_group", "diagnosis_category"])["icd_code"].count().reset_index()

# Rename columns for clarity
anchor_year_summary.rename(columns={"icd_code": "count"}, inplace=True)

# Display results
print(anchor_year_summary)


   anchor_year_group        diagnosis_category  count
0        2008 - 2010          Bipolar Disorder    562
1        2008 - 2010           Eating Disorder     26
2        2008 - 2010          Major Depression    744
3        2008 - 2010  Persistent Mood Disorder      6
4        2008 - 2010             Schizophrenia    189
5        2011 - 2013          Bipolar Disorder    340
6        2011 - 2013           Eating Disorder     36
7        2011 - 2013          Major Depression    514
8        2011 - 2013             Schizophrenia    106
9        2014 - 2016          Bipolar Disorder    509
10       2014 - 2016           Eating Disorder     47
11       2014 - 2016          Major Depression   1366
12       2014 - 2016  Persistent Mood Disorder      2
13       2014 - 2016             Schizophrenia    164
14       2017 - 2019          Bipolar Disorder    670
15       2017 - 2019           Eating Disorder     45
16       2017 - 2019          Major Depression   2020
17       2017 - 2019  Persis

In [8]:
df_primary_filtered.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,diagnosis_category,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10002315,22798410,1,F329,10,Major Depression,M,27,2161,2017 - 2019,NaN
1,10006630,27341635,1,F329,10,Major Depression,F,21,2176,2014 - 2016,NaN
2,10007218,27767127,1,F329,10,Major Depression,F,24,2113,2017 - 2019,NaN
3,10007218,28348073,1,F329,10,Major Depression,F,24,2113,2017 - 2019,NaN
4,10007266,27967951,1,F332,10,Major Depression,F,55,2189,2017 - 2019,NaN


In [9]:
# Load discharge_detail.csv (assuming it contains subject_id and discharge notes)
discharge_df = pd.read_csv("~/Desktop/discharge_detail.csv")[["subject_id", "note_id"]]

# Aggregate: Count number of notes per subject
discharge_counts = discharge_df.groupby("subject_id")["note_id"].count().reset_index()
discharge_counts.rename(columns={"note_id": "note_count"}, inplace=True)

discharge_counts.head()


,subject_id,note_count
0,10000032,4
1,10000084,1
2,10000117,2
3,10000248,1
4,10000560,1


In [10]:
# Merge with df_primary_filtered to get relevant diagnoses and discharge note counts
df_merged = df_primary_filtered.merge(discharge_counts, on="subject_id", how="left")

# Count unique patients per diagnosis category who have at least one discharge note
note_summary = (
    df_merged.dropna(subset=["note_count"])  # Keep only patients with discharge notes
    .groupby(["anchor_year_group", "diagnosis_category"])["subject_id"]
    .nunique()
    .reset_index()
)

# Rename columns for clarity
note_summary.rename(columns={"subject_id": "patient_count"}, inplace=True)

# Display results
print(note_summary)

   anchor_year_group        diagnosis_category  patient_count
0        2008 - 2010          Bipolar Disorder            112
1        2008 - 2010           Eating Disorder              8
2        2008 - 2010          Major Depression            203
3        2008 - 2010  Persistent Mood Disorder              3
4        2008 - 2010             Schizophrenia             34
5        2011 - 2013          Bipolar Disorder             59
6        2011 - 2013           Eating Disorder              4
7        2011 - 2013          Major Depression            116
8        2011 - 2013             Schizophrenia             18
9        2014 - 2016          Bipolar Disorder            117
10       2014 - 2016           Eating Disorder             10
11       2014 - 2016          Major Depression            274
12       2014 - 2016             Schizophrenia             38
13       2017 - 2019          Bipolar Disorder            127
14       2017 - 2019           Eating Disorder             11
15      

In [11]:
# Load the discharge notes dataset
discharge_notes_df = pd.read_csv(
    "/Users/lifuchen/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv.gz",
    compression="gzip"
)

# # Ensure correct column names (assuming columns include 'subject_id', 'hadm_id', and 'text')
# discharge_notes_df = discharge_notes_df[["subject_id", "hadm_id", "text"]]

# Merge with df_primary_filtered on both subject_id and hadm_id
df_merged = df_primary_filtered.merge(discharge_notes_df, on=["subject_id", "hadm_id"], how="left")

# Display a preview of the merged dataset
print(df_merged.head())


   subject_id   hadm_id  seq_num icd_code  icd_version diagnosis_category  \
0    10002315  22798410        1     F329           10   Major Depression   
1    10006630  27341635        1     F329           10   Major Depression   
2    10007218  27767127        1     F329           10   Major Depression   
3    10007218  28348073        1     F329           10   Major Depression   
4    10007266  27967951        1     F332           10   Major Depression   

  gender  anchor_age  anchor_year anchor_year_group  dod note_id note_type  \
0      M          27         2161       2017 - 2019  NaN     NaN       NaN   
1      F          21         2176       2014 - 2016  NaN     NaN       NaN   
2      F          24         2113       2017 - 2019  NaN     NaN       NaN   
3      F          24         2113       2017 - 2019  NaN     NaN       NaN   
4      F          55         2189       2017 - 2019  NaN     NaN       NaN   

   note_seq charttime storetime text  
0       NaN       NaN       N

In [12]:
# Count unique patients per diagnosis category who have at least one discharge note
note_summary = (
    df_merged.dropna(subset=["note_id"])  # Keep only patients with discharge notes
    .groupby(["anchor_year_group", "diagnosis_category"])["note_id"]
    .nunique()
    .reset_index()
)

# Rename columns for clarity
note_summary.rename(columns={"subject_id": "patient_count"}, inplace=True)

# Display results
print(note_summary)

   anchor_year_group        diagnosis_category  note_id
0        2008 - 2010          Bipolar Disorder      133
1        2008 - 2010           Eating Disorder       16
2        2008 - 2010          Major Depression      117
3        2008 - 2010  Persistent Mood Disorder        1
4        2008 - 2010             Schizophrenia       58
5        2011 - 2013          Bipolar Disorder       96
6        2011 - 2013           Eating Disorder       28
7        2011 - 2013          Major Depression       74
8        2011 - 2013             Schizophrenia       26
9        2014 - 2016          Bipolar Disorder      166
10       2014 - 2016           Eating Disorder       35
11       2014 - 2016          Major Depression      318
12       2014 - 2016  Persistent Mood Disorder        1
13       2014 - 2016             Schizophrenia       64
14       2017 - 2019          Bipolar Disorder      190
15       2017 - 2019           Eating Disorder       28
16       2017 - 2019          Major Depression  

In [13]:
# Get the first clinical note from the DataFrame
note_text = df_merged.dropna(subset=["note_id"])['text'].iloc[0]

# Display the formatted clinical note
print(note_text)


 
Name:  ___                    Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: PSYCHIATRY
 
Allergies: 
pcn / sulfa / phenobarbital / labetalol / ampicillin / 
amoxicillin
 
Attending: ___.
 
Chief Complaint:
"I'm jittery from the risperidone" 
 
Major Surgical or Invasive Procedure:
None
 
History of Present Illness:
Mr. ___ is a ___ year old man with self-reported psychiatric 
history of OCD, anxiety, depression, and somatization disorder, 
recently hospitalized on Deac4 for an unspecified psychotic 
episode that was possibly substance-induced (___), who 
self-presented to the ED due to concerns that his risperidone 
was making him increasingly restless and anxious. Psychiatry was 
consulted for evaluation due to ED resident concern of possible 
SI.
.
Please also see Dr. ___ comprehensive discharge 
summary dated ___ for full details of last Deac4 hospital 
course.  Briefly, at that time patient was hospi

In [14]:
df_merged["note_length"] = df_merged["text"].apply(lambda x: len(str(x)))  # Length in characters

note_length_summary = (
    df_merged.dropna(subset=["note_id"])
    .groupby(["anchor_year_group"])["note_length"]
    .mean()  # Calculate average length of notes
    .round(0)
    .astype(int)
    .reset_index()
)

note_length_summary.rename(columns={"note_length": "average_note_length"}, inplace=True)

print(note_length_summary)


  anchor_year_group  average_note_length
0       2008 - 2010                17773
1       2011 - 2013                16280
2       2014 - 2016                15771
3       2017 - 2019                18096


In [ ]:
note_length_summary = (
    df_merged.dropna(subset=["note_id"])
    .groupby(["diagnosis_category"])["note_length"]
    .mean()  # Calculate average length of notes
    .round(0)
    .astype(int)
    .reset_index()
)

note_length_summary.rename(columns={"note_length": "average_note_length"}, inplace=True)

print(note_length_summary)

         diagnosis_category  average_note_length
0          Bipolar Disorder                18052
1           Eating Disorder                12006
2          Major Depression                17013
3  Persistent Mood Disorder                17549
4             Schizophrenia                17165


In [16]:
note_length_summary = (
    df_merged.dropna(subset=["note_id"])
    .groupby(["anchor_year_group","diagnosis_category"])["note_length"]
    .mean()  # Calculate average length of notes
    .round(0)
    .astype(int)
    .reset_index()
)

note_length_summary.rename(columns={"note_length": "average_note_length"}, inplace=True)

print(note_length_summary)

   anchor_year_group        diagnosis_category  average_note_length
0        2008 - 2010          Bipolar Disorder                18577
1        2008 - 2010           Eating Disorder                14830
2        2008 - 2010          Major Depression                17749
3        2008 - 2010  Persistent Mood Disorder                16586
4        2008 - 2010             Schizophrenia                16808
5        2011 - 2013          Bipolar Disorder                17144
6        2011 - 2013           Eating Disorder                12764
7        2011 - 2013          Major Depression                16756
8        2011 - 2013             Schizophrenia                15521
9        2014 - 2016          Bipolar Disorder                16721
10       2014 - 2016           Eating Disorder                12163
11       2014 - 2016          Major Depression                15501
12       2014 - 2016  Persistent Mood Disorder                 9124
13       2014 - 2016             Schizophrenia  